In [1]:
import pandas as pd
import requests
import time

from etl_resources import sqlite_connection, get_api_key, get_symbol_list

In [2]:
def build_statements_dataset():
    
    '''
    This function uses the alphavantage api to pull financial statements for 
    S&P 100 equities. Balance sheets, cash flow, income statements, and earnings
    estimates are included.
    '''
    
    api_key = get_api_key()
    symbol_list = get_symbol_list()
    con = sqlite_connection()
    
    statements = {'INCOME_STATEMENT':'quarterlyReports',
                  'BALANCE_SHEET':'quarterlyReports',
                  'CASH_FLOW':'quarterlyReports'}
    
    for statement, filters in statements.items(): 
        
        for symbol in symbol_list:

            time.sleep(20) # Handle the rate limiter
            url = f"https://www.alphavantage.co/query?function={statement}&symbol={symbol}&apikey={api_key}"

            try:
                
                print(f"Parsing {statement} for {symbol}")
                response = requests.get(url)
                json = response.json()[f"{filters}"]
                df = pd.DataFrame.from_dict(json)
                df['ticker'] = symbol

                table_name = statement.lower() 
                df.to_sql(name=table_name, if_exists='append', index=False, con=con)

            except:
                # TODO: Better failure logging
                print(f"Failed: {url}")
 


In [3]:
def process_financials():
    
    con = sqlite_connection()
    
    tables = ['balance_sheet','cash_flow','income_statement']
    
    non_nums = ['fiscalDateEnding','reportedCurrency','ticker']
    
    for table in tables:
        print(f'Processing {table}')
        df = pd.read_sql(f"select * from {table}",con=con)
        
        nums = [col for col in df.columns if col not in non_nums]
        
        for field in nums:
            df[field] = pd.to_numeric(df[field],errors='coerce')
        
        df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
        
        df.columns = [col.lower().strip() for col in df.columns]
        
        df.to_sql(name=f"{table}_clean",con=con, if_exists='replace')
        
    

In [4]:
def process_timeseries_differences():
    
    con = sqlite_connection()
    cur = con.cursor()
    
    try:
        cur.execute("drop table balance_sheet_qtr")
    except:
        print("balance_sheet_qtr doesnt exist")
    
    
    cur.execute("select distinct ticker from balance_sheet_clean")
    res = cur.fetchall()
    res = [val[0] for val in res]
    
    for ticker in res:
        qry = f'''
    select b.*,c.quarter, c.year from balance_sheet_clean b

    left join calendar c on b.fiscaldateending = c.date
    where b.ticker='{ticker}'
    order by b.ticker, c.year, c.quarter
        '''

        df = pd.read_sql(qry,con=con)
        df = df.drop(columns=['reportedcurrency','index'])
        df = df.set_index(['fiscaldateending', 'quarter','year', 'ticker'])
        perc_df = df.pct_change()
        diff_df = df.fillna(0)
        diff_df = diff_df.diff()

        combined_df = pd.merge(perc_df, diff_df, on=['fiscaldateending','quarter','year','ticker'])
        combined_df.columns = [col.replace('_x','_pct').replace('_y','_val') for col in combined_df.columns]
        final_df = pd.merge(df,combined_df, on=['fiscaldateending','quarter','year','ticker'])

        final_df.to_sql(name='balance_sheet_qtr',if_exists='append', con=con)

In [5]:
#build_statements_dataset()
#process_financials()
process_timeseries_differences()

balance_sheet_qtr doesnt exist


,,,,totalassets,totalcurrentassets,cashandcashequivalentsatcarryingvalue,cashandshortterminvestments,inventory,currentnetreceivables,totalnoncurrentassets,propertyplantequipment,accumulateddepreciationamortizationppe,intangibleassets,...,currentlongtermdebt_val,longtermdebtnoncurrent_val,shortlongtermdebttotal_val,othercurrentliabilities_val,othernoncurrentliabilities_val,totalshareholderequity_val,treasurystock_val,retainedearnings_val,commonstock_val,commonstocksharesoutstanding_val
fiscaldateending,quarter,year,ticker,,,,,,,,,,,,,,,,,,,,,
2016-09-30 00:00:00,3,2016,AAPL,321686000000,106869000000,20484000000,6.715500e+10,2.132000e+09,2.929900e+10,3.852470e+11,2.701000e+10,3.423500e+10,8.620000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-31 00:00:00,4,2016,AAPL,331141000000,103332000000,16371000000,6.045200e+10,2.712000e+09,2.797700e+10,4.134470e+11,2.651000e+10,3.624900e+10,8.271000e+09,...,-1.000000e+06,-1.870000e+09,-7.698900e+10,-1.910000e+08,1.827000e+09,4.141000e+09,0.0,3.637000e+09,8.930000e+08,-8.074300e+07
2017-03-31 00:00:00,1,2017,AAPL,334532000000,101990000000,15157000000,1.515700e+10,2.910000e+09,NaN,2.325420e+11,2.716300e+10,3.796100e+10,8.090000e+09,...,5.000000e+08,1.097400e+10,1.081700e+10,-2.070000e+08,1.569000e+09,1.692000e+09,0.0,9.240000e+08,1.435000e+09,-4.960800e+07
2017-06-30 00:00:00,2,2017,AAPL,345173000000,112875000000,18571000000,1.857100e+10,3.146000e+09,NaN,2.322980e+11,2.928600e+10,3.969500e+10,8.105000e+09,...,2.496000e+09,5.333000e+09,9.817000e+09,-7.400000e+07,-8.720000e+08,-1.657000e+09,0.0,-2.400000e+09,8.660000e+08,-3.603300e+07
2017-09-30 00:00:00,3,2017,AAPL,375319000000,128645000000,20289000000,7.418100e+10,4.855000e+09,3.567300e+10,2.466740e+11,3.378300e+10,4.129300e+10,8.015000e+09,...,1.000000e+06,7.343000e+09,1.114550e+11,2.294300e+10,1.817000e+09,1.622000e+09,0.0,-1.950000e+08,1.422000e+09,-4.358100e+07
2017-12-31 00:00:00,4,2017,AAPL,406794000000,143810000000,27491000000,7.715300e+10,4.421000e+09,5.089900e+10,4.709280e+11,3.367900e+10,4.343100e+10,8.038000e+09,...,2.000000e+06,6.715000e+09,-9.702700e+10,-2.250700e+10,3.339000e+09,6.152000e+09,0.0,6.263000e+09,5.800000e+08,-4.455000e+07
2018-03-31 00:00:00,1,2018,AAPL,367502000000,130053000000,45059000000,8.794000e+10,7.662000e+09,2.240800e+10,4.167350e+11,3.507700e+10,4.542500e+10,NaN,...,2.000000e+09,-2.560000e+09,1.140000e+08,-2.690000e+08,3.101000e+09,-1.332100e+10,0.0,-1.269500e+10,1.597000e+09,-1.383690e+08
2018-06-30 00:00:00,2,2018,AAPL,349197000000,115761000000,31971000000,7.097000e+10,5.936000e+09,2.636700e+10,4.062090e+11,3.811700e+10,4.725100e+10,NaN,...,-3.000000e+09,-4.234000e+09,-7.010000e+09,-3.720000e+08,-1.161000e+09,-1.192900e+10,0.0,-1.246200e+10,5.800000e+08,-1.003650e+08
2018-09-30 00:00:00,3,2018,AAPL,365725000000,131339000000,25913000000,2.591300e+10,3.956000e+09,4.899500e+10,2.343860e+11,4.130400e+10,4.909900e+10,NaN,...,3.286000e+09,-3.393000e+09,1.042610e+11,2.592400e+10,3.220000e+09,-7.802000e+09,0.0,-9.036000e+09,1.577000e+09,-8.793100e+07


In [6]:
con = sqlite_connection()

res

['MMM',
 'BKNG',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'MO',
 'AMZN',
 'AXP',
 'AIG',
 'AMGN',
 'AAPL',
 'T',
 'BAC',
 'BK',
 'BIIB',
 'BLK',
 'BMY',
 'COF',
 'CAT',
 'CHTR',
 'CVX',
 'CSCO',
 'C',
 'CL',
 'CMCSA',
 'COP',
 'COST',
 'CVS',
 'DHR',
 'DUK',
 'LLY',
 'EMR',
 'EXC',
 'XOM',
 'FB',
 'FDX',
 'F',
 'GD',
 'GE',
 'GM',
 'GILD',
 'GOOG',
 'GOOGL',
 'HD',
 'HON',
 'INTC',
 'IBM',
 'JNJ',
 'JPM',
 'KMI',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDT',
 'MRK',
 'MET',
 'MSFT',
 'MDLZ',
 'MS',
 'NFLX',
 'NEE',
 'NKE',
 'NVDA',
 'OXY',
 'ORCL',
 'PYPL',
 'PEP',
 'PFE',
 'PM',
 'PG',
 'QCOM',
 'SLB',
 'SPG',
 'SO',
 'SBUX',
 'TGT',
 'TXN',
 'ALL',
 'BA',
 'KO',
 'DOW',
 'GS',
 'KHC',
 'UNP',
 'UPS',
 'UNH',
 'USB',
 'VZ',
 'V',
 'WMT',
 'WBA',
 'DIS',
 'WFC']